In [3]:


import os
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import librosa
import numpy as np

# Check GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)



# Dummy Dataset Generator
class DummyASVspoofDataset(Dataset):
    def __init__(self, num_samples=100):
        self.num_samples = num_samples

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        waveform = torch.randn(1, 16000)
        label = torch.tensor(idx % 2, dtype=torch.float32)
        return waveform, label

# RawNet2 Lite Model
class RawNet2Lite(nn.Module):
    def __init__(self):
        super(RawNet2Lite, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm1d(16)
        self.pool = nn.MaxPool1d(2)
        self.gru = nn.GRU(16, 32, batch_first=True)
        self.fc = nn.Linear(32, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = x.transpose(1, 2)
        _, h_n = self.gru(x)
        out = torch.sigmoid(self.fc(h_n.squeeze(0)))
        return out.squeeze()

# Training Function
def train(model, loader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        total_loss = 0.0
        for batch in loader:
            waveforms, labels = batch
            waveforms, labels = waveforms.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(waveforms)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(loader):.4f}")

# Evaluation Function
def evaluate(model, loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in loader:
            waveforms, labels = batch
            waveforms = waveforms.to(device)
            outputs = model(waveforms)
            preds = (outputs > 0.5).float().cpu()
            all_preds.extend(preds)
            all_labels.extend(labels)

    print(classification_report(all_labels, all_preds, target_names=["bonafide", "spoof"]))

# Initialize and Run

dataset = DummyASVspoofDataset()
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

model = RawNet2Lite().to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train(model, dataloader, criterion, optimizer, epochs=5)
evaluate(model, dataloader)


Using device: cpu
Epoch 1/5, Loss: 0.7009
Epoch 2/5, Loss: 0.6925
Epoch 3/5, Loss: 0.7035
Epoch 4/5, Loss: 0.6976
Epoch 5/5, Loss: 0.6928
              precision    recall  f1-score   support

    bonafide       0.45      0.56      0.50        50
       spoof       0.42      0.32      0.36        50

    accuracy                           0.44       100
   macro avg       0.44      0.44      0.43       100
weighted avg       0.44      0.44      0.43       100

